In [1]:
# GPU
# 计算设备
''' 
我们可以指定⽤于存储和计算的设备，如CPU和GPU。默认情况下，张量是在内存中创建的，然后使⽤CPU计
算它。
在PyTorch中，CPU和GPU可以⽤torch.device('cpu') 和torch.device('cuda')表⽰。应该注意的
是，cpu设备意味着所有物理CPU和内存，这意味着PyTorch的计算将尝试使⽤所有CPU核⼼。然⽽，gpu设备
只代表⼀个卡和相应的显存。如果有多个GPU，我们使⽤torch.device(f'cuda:{i}') 来表⽰第i块GPU
（i从0开始）。另外，cuda:0和cuda是等价的
'''
import torch
from torch import nn
# cuda:0和cuda是等价的
torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

In [3]:
'我们可以查询可⽤gpu的数量'
torch.cuda.device_count() # 查询gpu数量

1

In [4]:
# 现在我们定义了两个⽅便的函数，这两个函数允许我们在不存在所需所有GPU的情况下运⾏代码
# 即尝试测试是否有gpu,若有则返回gpu,无则返回cpu
def try_gpu(i=0): #@save
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')
def try_all_gpus(): #@save
    """返回所有可⽤的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
               for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]
try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

In [5]:
# 张量与gpu
''' 
我们可以查询张量所在的设备。默认情况下，张量是在CPU上创建的

需要注意的是，⽆论何时我们要对多个项进⾏操作，它们都必须在同⼀个设备上。例如，如果我们对两个张
量求和，我们需要确保两个张量都位于同⼀个设备上，否则框架将不知道在哪⾥存储结果，甚⾄不知道在哪
⾥执⾏计算
'''
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [6]:
# 存储在GPU上
''' 
有⼏种⽅法可以在GPU上存储张量。例如，我们可以在创建张量时指定存储设备。接下来，我们在第⼀个gpu上
创建张量变量X。在GPU上创建的张量只消耗这个GPU的显存。我们可以使⽤nvidia-smi命令查看显存使
⽤情况。⼀般来说，我们需要确保不创建超过GPU显存限制的数据
'''
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [9]:
!nvidia-smi

Thu Oct  6 11:25:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.63       Driver Version: 512.36       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
| N/A   37C    P5    19W /  N/A |   1757MiB /  8192MiB |     50%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [24]:
''' 
假设你⾄少有两个GPU，下⾯的代码将在第⼆个GPU上创建⼀个随机张量

'''
Y = torch.rand(2, 3, device=try_gpu())
Y
# Y.cuda() # 若没有GPU1,则Y存储到CPU中
# X+Y

tensor([[0.9447, 0.5018, 0.9690],
        [0.4654, 0.2505, 0.3227]], device='cuda:0')

In [28]:
# 复制
''' 
如果我们要计算X + Y，我们需要决定在哪⾥执⾏这个操作。例如，如 图5.6.1所⽰，我们可以将X传输到第
⼆个GPU并在那⾥执⾏操作。不要简单地X加上Y，因为这会导致异常，运⾏时引擎不知道该怎么做：它在同
⼀设备上找不到数据会导致失败。由于Y位于第⼆个GPU上，所以我们需要将X移到那⾥，然后才能执⾏相加
运算。
'''
Z = X.cuda() # cuda 即GPU
print(X)
print(Z)
X.cuda() is X # 因为X已经在GPU0 上,载体调用X.cuda()将会返回X,不会复制并分配新内存

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')


True

In [26]:
X+Z

tensor([[2., 2., 2.],
        [2., 2., 2.]], device='cuda:0')

In [ ]:
''' 
假设变量Z已经存在于第⼆个GPU上。如果我们还是调⽤Z.cuda(1)会发⽣什么？它将返回Z，⽽不会复制
并分配新内存

Z.cuda(1) is Z
True
'''


In [ ]:
# 旁注
''' 
⼈们使⽤GPU来进⾏机器学习，因为单个GPU相对运⾏速度快。但是在设备（CPU、GPU和其他机器）之间
传输数据⽐计算慢得多。这也使得并⾏化变得更加困难，因为我们必须等待数据被发送（或者接收），然后
才能继续进⾏更多的操作。这就是为什么拷⻉操作要格外⼩⼼。根据经验，多个⼩操作⽐⼀个⼤操作糟糕得
多。此外，⼀次执⾏⼏个操作⽐代码中散布的许多单个操作要好得多（除⾮你确信⾃⼰在做什么）。如果⼀个
设备必须等待另⼀个设备才能执⾏其他操作，那么这样的操作可能会阻塞。这有点像排队订购咖啡，⽽不像
通过电话预先订购：当你到店的时候，咖啡已经准备好了

最后，当我们打印张量或将张量转换为NumPy格式时，如果数据不在内存中，框架会⾸先将其复制到内存中，
这会导致额外的传输开销。更糟糕的是，它现在受制于全局解释器锁，使得⼀切都得等待Python完成。
'''

In [30]:
# 神经网络与GPU
'类似地，神经⽹络模型可以指定设备。下⾯的代码将模型参数放在GPU上'
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu()) # 指定设备,将模型代码参数放在GPU上

In [45]:
''' 
在接下来的⼏章中，我们将看到更多关于如何在GPU上运⾏模型的例⼦，因为它们将变得更加计算密集。
当输⼊为GPU上的张量时，模型将在同⼀GPU上计算结果
'''
C = torch.ones(2, 3).cuda()
# B= C.cuda()
net(C)

tensor([[-0.1733],
        [-0.1733]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [46]:
# 确认模型参数存储在同一个GPU上
net[0].weight.data.device

device(type='cuda', index=0)

In [ ]:
''' 
⼩结
• 我们可以指定⽤于存储和计算的设备，例如CPU或GPU。默认情况下，数据在主内存中创建，然后使
⽤CPU进⾏计算。
• 深度学习框架要求计算的所有输⼊数据都在同⼀设备上，⽆论是CPU还是GPU。
• 不经意地移动数据可能会显著降低性能。⼀个典型的错误如下：计算GPU上每个⼩批量的损失，并在
命令⾏中将其报告给⽤⼾（或将其记录在NumPy ndarray中）时，将触发全局解释器锁，从⽽使所
有GPU阻塞。最好是为GPU内部的⽇志分配内存，并且只移动较⼤的⽇志。
'''